# The "sync" package, Mutexes

## Imports

In [ ]:
import (
    "fmt"
    "sync"
    "time"
)

## Intro to sync.Mutex

One of the main mechanisms for managing state in Go is communication over channels, as we learned in the previous notebook. However, as stated in [this useful article](https://github.com/golang/go/wiki/MutexOrChannel) in the official Go wiki:

> A common Go newbie mistake is to over-use channels and goroutines just because it's possible, and/or because it's fun. Don't be afraid to use a `sync.Mutex` if that fits your problem best.

So what does that mean, and what the heck is a "Mutex"? Well, a mutex is a mutual exclusion lock that can be utilized as a rule, such that we can safely access data across multiple goroutines. More specifically, a Mutex allows us to Lock certain data (e.g., a struct) that may also be accessed by other goroutines, such that we ensure exclusive access to the data until we Unlock that data.

## Example

Let's say that we have our Stats struct from the first notebook. If we wanted to have the option to Lock/Unlock this Stats data, we might include a Mutex in the struct as follows:

In [ ]:
// Stats stores aggregated stats about
// tweets collected over time
type Stats struct {
    SentimentAverage  float64
    Counts            map[string]int
    Mux               sync.Mutex
}

Now, we will eventually want to update the `Counts`, for example. If we have multiple goroutines that might be updating these `Counts`, we can wrap the operation in an Lock and Unlock that references the Mutex:

In [ ]:
// IncrementCount the count of tweets.
func (s *Stats) IncrementCount(key string) {
    
    // Lock so only the current goroutine can access the map.
    s.Mux.Lock()
    
    // Increment the count.
    s.Counts[key]++
    
    // Unlock the data.
    s.Mux.Unlock()
}

We can also do this for accessing the current values in the Counts:

In [ ]:
// GetCount returns a count of tweets.
func (s *Stats) GetCount(key string) int {
    s.Mux.Lock()
    defer s.Mux.Unlock()
    return s.Counts[key]
}

Now, let's try spinning up a bunch of goroutines that are trying to increment these counts at the same time. The Mutex will ensure that we do this safely:

In [ ]:
// Initialize our tweet stats.
stats := &Stats{
    Counts: map[string]int{
        "positive": 0,
        "negative": 0,
        "neutral": 0,
    },
    Mux: sync.Mutex{},
}

In [ ]:
for i := 0; i < 100; i++ {
    go stats.IncrementCount("positive")
}

time.Sleep(time.Second)
fmt.Println(stats.GetCount("positive"))

## Channels vs. Mutexes

There isn't any strict rule about when you should use channels to communicate data (i.e., share state) and when you should use Mutexes to manage state. However, we do have some guidance from the Go team [here](https://github.com/golang/go/wiki/MutexOrChannel):

> Use whichever is most expressive and/or most simple.
> ...
> If you ever find your sync.Mutex locking rules are getting too complex, ask yourself whether using channel(s) might be simpler.

So, we should keep simplicity in mind and not always reach for channels. In fact, if you look at large, established Go projects on GitHub, you might be surprised to see more Mutexes and fewer channels than expected. 